In [1]:
!pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl pacmap datasets langchain-community ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 20.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd

In [3]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset

In [24]:
df = pd.read_csv('/kaggle/input/resume-dataset/Resume/Resume.csv')


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [6]:
# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

In [10]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["Resume_str"], metadata={"Category": doc["Category"]}) for doc in tqdm(dataset)
]

  0%|          | 0/2484 [00:00<?, ?it/s]

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])

In [12]:
from sentence_transformers import SentenceTransformer

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter
print(f"Model's maximum sequence length: {SentenceTransformer('thenlper/gte-small').max_seq_length}")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]

2025-05-20 07:44:54.671179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747727094.852425      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747727094.905318      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model's maximum sequence length: 512


  0%|          | 0/19489 [00:00<?, ?it/s]

In [13]:
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [14]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

/tmp/ipykernel_31/2775243909.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
2025-05-20 07:45:46.226050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747727146.247622     125 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747727146.254250     125 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has 

Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
KNOWLEDGE_VECTOR_DATABASE

In [16]:
# retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever()
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever(search_kwargs={"k": 5})

In [20]:
docs = retriever.invoke("""
  "Machine Learning Engineer" OR "Data Scientist" AND (
    ("Python" AND ("Scikit-learn" OR "TensorFlow" OR "PyTorch"))  
    ("SQL" AND ("Spark" OR "Hadoop" OR "ETL"))  
    ("AWS" OR "GCP" OR "Azure" OR "MLOps")  
    ("Tableau" OR "Power BI" OR "data visualization")  
    ("statistical analysis" OR "A/B testing")  
    ("NLP" OR "LLM" OR "GenAI" OR "recommendation systems")  
  )  
  NOT ("intern" OR "student")  
  Years: "3+ years"  
  Location: "Remote" OR "Mumbai" OR "Bangalore" OR "Pune"  
""")

2025-05-20 08:01:14.105384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747728074.127330     171 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747728074.134129     171 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
# docs = retriever.invoke("""**Job Duties:**  
# - Develop and deploy **machine learning models** to solve business problems.  
# - Clean, process, and analyze **large datasets** to extract actionable insights.  
# - Collaborate with **cross-functional teams** (engineering, product, marketing) to implement data-driven solutions.  
# - Design and maintain **data pipelines** for efficient data collection and processing.  
# - Conduct **statistical analysis and A/B testing** to measure the impact of business decisions.  
# - Build **predictive models and recommendation systems** to enhance user experience.  
# - Create **data visualizations and dashboards** to communicate findings to stakeholders.  
# - Stay updated with the latest advancements in **AI/ML** and apply best practices.  
# - Automate data workflows to improve **efficiency and scalability**.  
# - Perform other **ad-hoc data-related tasks** as required.  

# **Required Skills & Qualifications:**  
# - **Technical Skills:**  
#   - Strong proficiency in **Python (Pandas, NumPy, Scikit-learn, TensorFlow/PyTorch)**.  
#   - Experience with **SQL** and big data tools (Spark, Hadoop).  
#   - Knowledge of **statistics, machine learning, and deep learning** techniques.  
#   - Familiarity with **data visualization tools** (Tableau, Power BI, Matplotlib/Seaborn).  
#   - Experience with **cloud platforms** (AWS, GCP, or Azure) and MLOps tools.  
# - **Soft Skills:**  
#   - Strong **problem-solving and analytical thinking**.  
#   - Ability to communicate complex findings to **non-technical stakeholders**.  
#   - Team collaboration and **project management** skills.  

# **Experience:**  
# - **3+ years** of hands-on experience in **data science, machine learning, or analytics**.  
# - Proven track record of **deploying ML models in production**.  
# - Experience with **end-to-end data pipelines** (ETL, feature engineering, model deployment).  
# - Prior work in **recommendation systems, NLP, or computer vision** is a plus.  

# **Location:**  
# - **Hybrid/Remote** (with occasional office visits if required).  
# - Preferred locations: **Mumbai, Bangalore, or Pune** (open to remote for strong candidates).  

# **Additional Preferences:**  
# - Experience with **GenAI/LLMs** (e.g., OpenAI, LangChain) is a **big plus**.  
# - Knowledge of **SAP HANA, R, or T-SQL** (for roles involving enterprise data).  
# """)

2025-05-20 07:46:54.813415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747727214.836039     147 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747727214.842858     147 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
import pprint

In [22]:
pprint.pp(docs)

[Document(id='8f268142-54cf-4526-9c17-f9f23b1f9c53', metadata={'Category': 'AUTOMOBILE', 'start_index': 5994}, page_content="Datastage jobs for small business requirements.          Education and Training        BACHELOR OF TECHNOLOGY   2011     ANNA UNIVERSITY   －   City  ,   State  ,   INDIA            BUSINESS INTELLIGENCE TRAINING   2011     TATA CONSULTANCY SERVICES   －   City  ,   State  ,   INDIA     SEPTEMBER 2011 - DECEMBER 2011\xa0   This course provides an overview that gives business and information technology professionals the confidence to dive right into their business intelligence and data warehousing activities. Hands-On training provided on ETL tools Informatica/Datastage and\xa0data warehousing environment for 90 days.          DATASTAGE TRAINING   2012     GREENS TECHNOLOGIES   －   City  ,   State  ,   INDIA   This course is designed to introduce ETL developers to Datastage Development, Data Warehousing and Data Modeling training's with real-world ETL process implem